# Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os
import csv
import cv2 as cv
if IS_COLAB:
    !pip install tensorflow_addons --quiet
    from google.colab import drive
import tensorflow_addons as tfa
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import gc
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from enum import Enum
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization, Activation
from keras.initializers import HeUniform, LecunUniform

from datetime import datetime
from functools import partial
from tensorflow.python.keras.utils.data_utils import Sequence

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

!nvidia-smi -q -i 0 | grep "Product Name"

sklearn version:  1.0.2
TF version:  2.8.2
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs
    Product Name                          : Tesla P100-PCIE-16GB


# Get the data

In [2]:
if IS_COLAB:
    drive.mount("/content/drive")
    location = "drive/MyDrive/Colab Notebooks/"
else:
    location = "./"
# 작성자 전민재, hsh
def load_Emist(exsitNumpy=False, needTranspose=True):
    if exsitNumpy == False:
        # "./emnist-byclass-test.csv"
        # train-set
        # "공용/datasets/Emnist/" 위치에 csv 저장
        csv_train_data_file = open(
            location + "datasets/Emnist/train_data.csv",
            "r",
            encoding="ms932",
            errors="",
            newline="",
        )  # 리스트 형식
        csv_train_label_file = open(
            location + "datasets/Emnist/train_label.csv",
            "r",
            encoding="ms932",
            errors="",
            newline="",
        )  # 리스트 형식
        f_train = csv.reader(
            csv_train_data_file,
            delimiter=",",
            doublequote=True,
            lineterminator="\r\n",
            quotechar='"',
            skipinitialspace=True,
        )
        l_train = csv.reader(
            csv_train_label_file,
            delimiter=",",
            doublequote=True,
            lineterminator="\r\n",
            quotechar='"',
            skipinitialspace=True,
        )
        # valid-set
        csv_valid_data_file = open(
            location + "datasets/Emnist/valid_data.csv",
            "r",
            encoding="ms932",
            errors="",
            newline="",
        )  # 리스트 형식
        csv_valid_label_file = open(
            location + "datasets/Emnist/valid_label.csv",
            "r",
            encoding="ms932",
            errors="",
            newline="",
        )  # 리스트 형식
        f_valid = csv.reader(
            csv_valid_data_file,
            delimiter=",",
            doublequote=True,
            lineterminator="\r\n",
            quotechar='"',
            skipinitialspace=True,
        )
        l_valid = csv.reader(
            csv_valid_label_file,
            delimiter=",",
            doublequote=True,
            lineterminator="\r\n",
            quotechar='"',
            skipinitialspace=True,
        )
        X_train = []
        y_train = []
        X_valid = []
        y_valid = []
        for i, row in enumerate(f_train):
            # 행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx] = int(char)
            # train
            # data 추가
            X_train.append(row)
        for i, row in enumerate(f_valid):
            # 행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx] = int(char)
            # valid
            # data 추가
            X_valid.append(row)
        for i, row in enumerate(l_train):
            # train
            # label 추가
            y_train.append(int(row[0]))
        for i, row in enumerate(l_valid):
            # valid
            # label 추가
            y_valid.append(int(row[0]))
        # train
        X_train = np.array(X_train, dtype=np.uint8)
        X_train = X_train.reshape(-1, 28, 28)
        # valid
        X_valid = np.array(X_valid, dtype=np.uint8)
        X_valid = X_valid.reshape(-1, 28, 28)
        csv_train_data_file.close()
        csv_train_label_file.close()
        csv_valid_data_file.close()
        csv_valid_label_file.close()
        # kaggle dataset이 시계반대방향으로 90도 회전 되있고 상하 반전 되어있음
        def rotate_90(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[c][N - 1 - r] = m[r][c]
            return np.array(ret, dtype=np.uint8)

        def vreflect(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[r][c] = m[r][N - 1 - c]
            return np.array(ret, dtype=np.uint8)

        if needTranspose == True:
            # train
            # 회전
            for idx, i in enumerate(X_train):
                X_train[idx] = rotate_90(i)
            # 상하반전
            for idx, i in enumerate(X_train):
                X_train[idx] = vreflect(i)
            # valid
            # 회전
            for idx, i in enumerate(X_valid):
                X_valid[idx] = rotate_90(i)
            # 상하반전
            for idx, i in enumerate(X_valid):
                X_valid[idx] = vreflect(i)
        # 저장용
        # train
        # (533917,28,28)
        np.save(location + "datasets/Emnist/X_train", np.array(X_train, dtype=np.uint8))
        np.save(location + "datasets/Emnist/y_train", np.array(y_train, dtype=np.uint8))
        # valid
        np.save(location + "datasets/Emnist/X_valid", np.array(X_valid, dtype=np.uint8))
        np.save(location + "datasets/Emnist/y_valid", np.array(y_valid, dtype=np.uint8))
    X_train = np.load(location + "datasets/Emnist/X_train.npy")
    y_train = np.load(location + "datasets/Emnist/y_train.npy")
    X_valid = np.load(location + "datasets/Emnist/X_valid.npy")
    y_valid = np.load(location + "datasets/Emnist/y_valid.npy")
    return X_train, y_train, X_valid, y_valid

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 황성현
# npy 파일이 존재할 경우 exsitNumpy = True, 없으면 False
X_train, y_train, X_valid, y_valid = load_Emist(exsitNumpy=True)
X_train = tf.reshape(X_train, [-1, 28, 28, 1])
X_valid = tf.reshape(X_valid, [-1, 28, 28, 1])
X_train_resize = np.uint8(tf.image.resize(X_train, [32, 32]))
X_valid_resize = np.uint8(tf.image.resize(X_valid, [32, 32]))

# Train the model

In [ ]:
# Model from Lecture
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="lecun_normal", input_shape=(32, 32, 1)), #He_normal have a problem with some learning rate scheduler
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="lecun_normal"),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="lecun_normal"),
            Flatten(),
            Dense(84, activation="relu", kernel_initializer="lecun_normal"),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 69s 4ms/step - loss: 1.2695 - accuracy: 0.6583 - val_loss: 0.5500 - val_accuracy: 0.8173
Epoch 2/100
16685/16685 [==============================] - 68s 4ms/step - loss: 0.6197 - accuracy: 0.8043 - val_loss: 0.5205 - val_accuracy: 0.8252
Epoch 3/100
16685/16685 [==============================] - 68s 4ms/step - loss: 0.5836 - accuracy: 0.8133 - val_loss: 0.5304 - val_accuracy: 0.8251
Epoch 4/100
16685/16685 [==============================] - 68s 4ms/step - loss: 0.5710 - accuracy: 0.8175 - val_loss: 0.5161 - val_accuracy: 0.8289
Epoch 5/100
16685/16685 [==============================] - 69s 4ms/step - loss: 0.5625 - accuracy: 0.8200 - val_loss: 0.5144 - val_accuracy: 0.8293
Epoch 6/100
16685/16685 [==============================] - 68s 4ms/step - loss: 0.5611 - accuracy: 0.8207 - val_loss: 0.5238 - val_accuracy: 0.8256
Epoch 7/100
16685/16685 [==============================] - 68s 4ms/step - loss: 0.5586 - accuracy: 0.8214 - val_

# Compare optimizers

In [ ]:
model = lenet_model()
model.compile(
    optimizer="SGD",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_SGD%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_SGD_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_SGD_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 77s 4ms/step - loss: 1.3436 - accuracy: 0.6318 - val_loss: 0.6615 - val_accuracy: 0.7843
Epoch 2/100
16685/16685 [==============================] - 66s 4ms/step - loss: 0.8042 - accuracy: 0.7528 - val_loss: 0.6124 - val_accuracy: 0.8007
Epoch 3/100
16685/16685 [==============================] - 65s 4ms/step - loss: 0.7235 - accuracy: 0.7749 - val_loss: 0.6076 - val_accuracy: 0.7906
Epoch 4/100
16685/16685 [==============================] - 67s 4ms/step - loss: 0.6791 - accuracy: 0.7864 - val_loss: 0.5509 - val_accuracy: 0.8170
Epoch 5/100
16685/16685 [==============================] - 66s 4ms/step - loss: 0.6502 - accuracy: 0.7942 - val_loss: 0.5387 - val_accuracy: 0.8182
Epoch 6/100
16685/16685 [==============================] - 66s 4ms/step - loss: 0.6309 - accuracy: 0.7996 - val_loss: 0.5326 - val_accuracy: 0.8215
Epoch 7/100
16685/16685 [==============================] - 66s 4ms/step - loss: 0.6136 - accuracy: 0.8041 - val_

In [ ]:
model = lenet_model()
model.compile(
    optimizer="RMSprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_RMSprop%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_RMSprop_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_RMSprop_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 76s 4ms/step - loss: 1.7505 - accuracy: 0.5647 - val_loss: 1.0292 - val_accuracy: 0.7292
Epoch 2/100
16685/16685 [==============================] - 77s 5ms/step - loss: 1.5106 - accuracy: 0.6269 - val_loss: 1.1443 - val_accuracy: 0.6940
Epoch 3/100
16685/16685 [==============================] - 74s 4ms/step - loss: 1.5425 - accuracy: 0.6144 - val_loss: 1.2517 - val_accuracy: 0.6604
Epoch 4/100
16685/16685 [==============================] - 76s 5ms/step - loss: 1.7622 - accuracy: 0.5747 - val_loss: 1.4949 - val_accuracy: 0.6309
Epoch 5/100
16685/16685 [==============================] - 75s 4ms/step - loss: 2.1736 - accuracy: 0.4976 - val_loss: 2.4907 - val_accuracy: 0.5011
Epoch 6/100
16685/16685 [==============================] - 76s 5ms/step - loss: 2.7300 - accuracy: 0.3911 - val_loss: 2.9100 - val_accuracy: 0.5152
Epoch 7/100
16685/16685 [==============================] - 75s 4ms/step - loss: 3.5343 - accuracy: 0.2038 - val_

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Adadelta",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_Adadelta%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Adadelta_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Adadelta_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 70s 4ms/step - loss: 27.9186 - accuracy: 0.0256 - val_loss: 4.4938 - val_accuracy: 0.0385
Epoch 2/100
16685/16685 [==============================] - 69s 4ms/step - loss: 4.3028 - accuracy: 0.0454 - val_loss: 4.0778 - val_accuracy: 0.0557
Epoch 3/100
16685/16685 [==============================] - 70s 4ms/step - loss: 4.0959 - accuracy: 0.0504 - val_loss: 4.0082 - val_accuracy: 0.0627
Epoch 4/100
16685/16685 [==============================] - 69s 4ms/step - loss: 4.0370 - accuracy: 0.0515 - val_loss: 3.9242 - val_accuracy: 0.0784
Epoch 5/100
16685/16685 [==============================] - 68s 4ms/step - loss: 3.9788 - accuracy: 0.0653 - val_loss: 3.8309 - val_accuracy: 0.1336
Epoch 6/100
16685/16685 [==============================] - 70s 4ms/step - loss: 3.9166 - accuracy: 0.0981 - val_loss: 3.7303 - val_accuracy: 0.1518
Epoch 7/100
16685/16685 [==============================] - 68s 4ms/step - loss: 3.8526 - accuracy: 0.1072 - val

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Adagrad",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_Adagrad%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Adagrad_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Adagrad_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 68s 4ms/step - loss: 3.7802 - accuracy: 0.1510 - val_loss: 2.9439 - val_accuracy: 0.3261
Epoch 2/100
16685/16685 [==============================] - 68s 4ms/step - loss: 3.1734 - accuracy: 0.2452 - val_loss: 2.4903 - val_accuracy: 0.4293
Epoch 3/100
16685/16685 [==============================] - 69s 4ms/step - loss: 2.9237 - accuracy: 0.2866 - val_loss: 2.2554 - val_accuracy: 0.4689
Epoch 4/100
16685/16685 [==============================] - 68s 4ms/step - loss: 2.7668 - accuracy: 0.3055 - val_loss: 2.0985 - val_accuracy: 0.5027
Epoch 5/100
16685/16685 [==============================] - 68s 4ms/step - loss: 2.6585 - accuracy: 0.3247 - val_loss: 2.0080 - val_accuracy: 0.5206
Epoch 6/100
16685/16685 [==============================] - 68s 4ms/step - loss: 2.5790 - accuracy: 0.3403 - val_loss: 1.9256 - val_accuracy: 0.5331
Epoch 7/100
16685/16685 [==============================] - 67s 4ms/step - loss: 2.5091 - accuracy: 0.3521 - val_

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Adamax",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_Adamax%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Adamax_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Adamax_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 71s 4ms/step - loss: 1.5342 - accuracy: 0.6010 - val_loss: 0.6161 - val_accuracy: 0.7978
Epoch 2/100
16685/16685 [==============================] - 70s 4ms/step - loss: 0.7283 - accuracy: 0.7742 - val_loss: 0.5097 - val_accuracy: 0.8262
Epoch 3/100
16685/16685 [==============================] - 70s 4ms/step - loss: 0.6065 - accuracy: 0.8067 - val_loss: 0.4802 - val_accuracy: 0.8358
Epoch 4/100
16685/16685 [==============================] - 70s 4ms/step - loss: 0.5494 - accuracy: 0.8219 - val_loss: 0.4553 - val_accuracy: 0.8416
Epoch 5/100
16685/16685 [==============================] - 69s 4ms/step - loss: 0.5185 - accuracy: 0.8296 - val_loss: 0.4481 - val_accuracy: 0.8451
Epoch 6/100
16685/16685 [==============================] - 79s 5ms/step - loss: 0.4979 - accuracy: 0.8355 - val_loss: 0.4500 - val_accuracy: 0.8451
Epoch 7/100
16685/16685 [==============================] - 70s 4ms/step - loss: 0.4822 - accuracy: 0.8391 - val_

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Nadam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_Nadam%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Nadam_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Nadam_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 90s 5ms/step - loss: 1.3897 - accuracy: 0.6361 - val_loss: 0.5482 - val_accuracy: 0.8185
Epoch 2/100
16685/16685 [==============================] - 90s 5ms/step - loss: 0.6145 - accuracy: 0.8070 - val_loss: 0.5133 - val_accuracy: 0.8296
Epoch 3/100
16685/16685 [==============================] - 89s 5ms/step - loss: 0.5822 - accuracy: 0.8155 - val_loss: 0.5006 - val_accuracy: 0.8363
Epoch 4/100
16685/16685 [==============================] - 89s 5ms/step - loss: 0.5674 - accuracy: 0.8203 - val_loss: 0.5104 - val_accuracy: 0.8355
Epoch 5/100
16685/16685 [==============================] - 89s 5ms/step - loss: 0.5650 - accuracy: 0.8210 - val_loss: 0.5369 - val_accuracy: 0.8261
Epoch 6/100
16685/16685 [==============================] - 88s 5ms/step - loss: 0.5605 - accuracy: 0.8223 - val_loss: 0.5070 - val_accuracy: 0.8313
Epoch 7/100
16685/16685 [==============================] - 89s 5ms/step - loss: 0.5565 - accuracy: 0.8233 - val_

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Ftrl",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_Ftrl%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Ftrl_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_Ftrl_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 72s 4ms/step - loss: 2.3586 - accuracy: 0.3900 - val_loss: 1.3400 - val_accuracy: 0.6304
Epoch 2/100
16685/16685 [==============================] - 73s 4ms/step - loss: 1.7451 - accuracy: 0.5157 - val_loss: 1.1047 - val_accuracy: 0.6793
Epoch 3/100
16685/16685 [==============================] - 72s 4ms/step - loss: 1.5658 - accuracy: 0.5562 - val_loss: 0.9900 - val_accuracy: 0.7020
Epoch 4/100
16685/16685 [==============================] - 73s 4ms/step - loss: 1.4599 - accuracy: 0.5798 - val_loss: 0.9126 - val_accuracy: 0.7213
Epoch 5/100
16685/16685 [==============================] - 75s 4ms/step - loss: 1.3876 - accuracy: 0.5974 - val_loss: 0.8704 - val_accuracy: 0.7330
Epoch 6/100
16685/16685 [==============================] - 73s 4ms/step - loss: 1.3324 - accuracy: 0.6109 - val_loss: 0.8327 - val_accuracy: 0.7420
Epoch 7/100
16685/16685 [==============================] - 72s 4ms/step - loss: 1.2892 - accuracy: 0.6214 - val_

Summary: adam is best

# Compare kernel initializers



In [ ]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="lecun_uniform"),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="lecun_uniform"),
            Flatten(),
            Dense(84, activation="relu", kernel_initializer="lecun_uniform"),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_lecun_uniform%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_lecun_uniform_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_lecun_uniform_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 96s 6ms/step - loss: 1.1991 - accuracy: 0.6740 - val_loss: 0.5596 - val_accuracy: 0.8122
Epoch 2/100
16685/16685 [==============================] - 73s 4ms/step - loss: 0.6183 - accuracy: 0.8053 - val_loss: 0.5228 - val_accuracy: 0.8267
Epoch 3/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.5826 - accuracy: 0.8153 - val_loss: 0.5131 - val_accuracy: 0.8267
Epoch 4/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.5699 - accuracy: 0.8192 - val_loss: 0.5166 - val_accuracy: 0.8326
Epoch 5/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.5644 - accuracy: 0.8211 - val_loss: 0.5268 - val_accuracy: 0.8248
Epoch 6/100
16685/16685 [==============================] - 70s 4ms/step - loss: 0.5596 - accuracy: 0.8226 - val_loss: 0.5391 - val_accuracy: 0.8246
Epoch 7/100
16685/16685 [==============================] - 69s 4ms/step - loss: 0.5602 - accuracy: 0.8222 - val_

In [ ]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="he_normal", input_shape=(32, 32, 1)),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="he_normal"),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="he_normal"),
            Flatten(),
            Dense(84, activation="relu", kernel_initializer="he_normal"),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_he_normal%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_he_normal_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_he_normal_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 74s 4ms/step - loss: 2.1167 - accuracy: 0.4725 - val_loss: 0.6154 - val_accuracy: 0.8026
Epoch 2/100
16685/16685 [==============================] - 72s 4ms/step - loss: 0.6768 - accuracy: 0.7917 - val_loss: 0.5402 - val_accuracy: 0.8215
Epoch 3/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.6076 - accuracy: 0.8088 - val_loss: 0.5238 - val_accuracy: 0.8275
Epoch 4/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.5819 - accuracy: 0.8160 - val_loss: 0.5251 - val_accuracy: 0.8283
Epoch 5/100
16685/16685 [==============================] - 73s 4ms/step - loss: 0.5707 - accuracy: 0.8186 - val_loss: 0.5431 - val_accuracy: 0.8256
Epoch 6/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.5653 - accuracy: 0.8196 - val_loss: 0.5407 - val_accuracy: 0.8263
Epoch 7/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.5637 - accuracy: 0.8201 - val_

In [ ]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="he_uniform", input_shape=(32, 32, 1)),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="he_uniform"),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, activation="relu", padding="valid", kernel_initializer="he_uniform"),
            Flatten(),
            Dense(84, activation="relu", kernel_initializer="he_uniform"),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_he_uniform%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_he_uniform_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_he_uniform_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 90s 5ms/step - loss: 2.7788 - accuracy: 0.3556 - val_loss: 0.6709 - val_accuracy: 0.7862
Epoch 2/100
16685/16685 [==============================] - 72s 4ms/step - loss: 0.7230 - accuracy: 0.7794 - val_loss: 0.5620 - val_accuracy: 0.8147
Epoch 3/100
16685/16685 [==============================] - 72s 4ms/step - loss: 0.6304 - accuracy: 0.8020 - val_loss: 0.5242 - val_accuracy: 0.8247
Epoch 4/100
16685/16685 [==============================] - 81s 5ms/step - loss: 0.6008 - accuracy: 0.8093 - val_loss: 0.5403 - val_accuracy: 0.8214
Epoch 5/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.5869 - accuracy: 0.8141 - val_loss: 0.5160 - val_accuracy: 0.8274
Epoch 6/100
16685/16685 [==============================] - 71s 4ms/step - loss: 0.5792 - accuracy: 0.8162 - val_loss: 0.5585 - val_accuracy: 0.8200
Epoch 7/100
16685/16685 [==============================] - 70s 4ms/step - loss: 0.5775 - accuracy: 0.8176 - val_

# Batch Normalization

In [ ]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [ ]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("lenet_BN%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_BN_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_BN_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 89s 5ms/step - loss: 0.7065 - accuracy: 0.7793 - val_loss: 0.4404 - val_accuracy: 0.8432
Epoch 2/100
16685/16685 [==============================] - 89s 5ms/step - loss: 0.5354 - accuracy: 0.8213 - val_loss: 0.4151 - val_accuracy: 0.8496
Epoch 3/100
16685/16685 [==============================] - 98s 6ms/step - loss: 0.5072 - accuracy: 0.8287 - val_loss: 0.4092 - val_accuracy: 0.8518
Epoch 4/100
16685/16685 [==============================] - 90s 5ms/step - loss: 0.4903 - accuracy: 0.8334 - val_loss: 0.3961 - val_accuracy: 0.8566
Epoch 5/100
16685/16685 [==============================] - 88s 5ms/step - loss: 0.4786 - accuracy: 0.8365 - val_loss: 0.3969 - val_accuracy: 0.8565
Epoch 6/100
16685/16685 [==============================] - 88s 5ms/step - loss: 0.4707 - accuracy: 0.8386 - val_loss: 0.3893 - val_accuracy: 0.8578
Epoch 7/100
16685/16685 [==============================] - 90s 5ms/step - loss: 0.4644 - accuracy: 0.8402 - val_

# Pad instead of Resize

In [3]:
X_train, y_train, X_valid, y_valid = load_Emist(exsitNumpy=True)
X_train = tf.reshape(X_train, [-1, 28, 28, 1])
X_valid = tf.reshape(X_valid, [-1, 28, 28, 1])
X_train_pad = np.uint8(tf.image.pad_to_bounding_box(X_train, 2, 2, 32, 32))
X_valid_pad = np.uint8(tf.image.pad_to_bounding_box(X_valid, 2, 2, 32, 32))

In [4]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [6]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_pad"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_pad_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_pad_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 111s 6ms/step - loss: 0.7013 - accuracy: 0.7810 - val_loss: 0.4333 - val_accuracy: 0.8458
Epoch 2/100
16685/16685 [==============================] - 83s 5ms/step - loss: 0.5274 - accuracy: 0.8237 - val_loss: 0.4093 - val_accuracy: 0.8522
Epoch 3/100
16685/16685 [==============================] - 82s 5ms/step - loss: 0.4982 - accuracy: 0.8318 - val_loss: 0.4030 - val_accuracy: 0.8539
Epoch 4/100
16685/16685 [==============================] - 82s 5ms/step - loss: 0.4817 - accuracy: 0.8359 - val_loss: 0.3913 - val_accuracy: 0.8571
Epoch 5/100
16685/16685 [==============================] - 81s 5ms/step - loss: 0.4698 - accuracy: 0.8391 - val_loss: 0.3883 - val_accuracy: 0.8586
Epoch 6/100
16685/16685 [==============================] - 81s 5ms/step - loss: 0.4613 - accuracy: 0.8414 - val_loss: 0.3862 - val_accuracy: 0.8587
Epoch 7/100
16685/16685 [==============================] - 82s 5ms/step - loss: 0.4565 - accuracy: 0.8427 - val

# Kernel Regularizer

In [7]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", kernel_regularizer="l1"),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [8]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_l1_on_C2"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_l1_on_C2_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_l1_on_C2_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 91s 5ms/step - loss: 0.9323 - accuracy: 0.7656 - val_loss: 0.6602 - val_accuracy: 0.8222
Epoch 2/100
16685/16685 [==============================] - 92s 6ms/step - loss: 0.7342 - accuracy: 0.8060 - val_loss: 0.6206 - val_accuracy: 0.8303
Epoch 3/100
16685/16685 [==============================] - 83s 5ms/step - loss: 0.7025 - accuracy: 0.8142 - val_loss: 0.5773 - val_accuracy: 0.8432
Epoch 4/100
16685/16685 [==============================] - 93s 6ms/step - loss: 0.6853 - accuracy: 0.8179 - val_loss: 0.5598 - val_accuracy: 0.8471
Epoch 5/100
16685/16685 [==============================] - 92s 6ms/step - loss: 0.6707 - accuracy: 0.8205 - val_loss: 0.5879 - val_accuracy: 0.8367
Epoch 6/100
16685/16685 [==============================] - 93s 6ms/step - loss: 0.6608 - accuracy: 0.8227 - val_loss: 0.5643 - val_accuracy: 0.8466
Epoch 7/100
16685/16685 [==============================] - 83s 5ms/step - loss: 0.6511 - accuracy: 0.8247 - val_

In [9]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", kernel_regularizer="l2"),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [11]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_l2_on_C2"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_l2_on_C2_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_l2_on_C2_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 108s 6ms/step - loss: 0.7607 - accuracy: 0.7756 - val_loss: 0.4779 - val_accuracy: 0.8401
Epoch 2/100
16685/16685 [==============================] - 84s 5ms/step - loss: 0.5867 - accuracy: 0.8159 - val_loss: 0.4737 - val_accuracy: 0.8426
Epoch 3/100
16685/16685 [==============================] - 94s 6ms/step - loss: 0.5598 - accuracy: 0.8232 - val_loss: 0.4673 - val_accuracy: 0.8437
Epoch 4/100
16685/16685 [==============================] - 83s 5ms/step - loss: 0.5448 - accuracy: 0.8269 - val_loss: 0.4526 - val_accuracy: 0.8466
Epoch 5/100
16685/16685 [==============================] - 83s 5ms/step - loss: 0.5352 - accuracy: 0.8299 - val_loss: 0.4434 - val_accuracy: 0.8505
Epoch 6/100
16685/16685 [==============================] - 93s 6ms/step - loss: 0.5277 - accuracy: 0.8316 - val_loss: 0.4364 - val_accuracy: 0.8535
Epoch 7/100
16685/16685 [==============================] - 94s 6ms/step - loss: 0.5224 - accuracy: 0.8329 - val

In [12]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform", kernel_regularizer="l1"),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [13]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_l1_on_FC"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_l1_on_FC_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_l1_on_FC_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 84s 5ms/step - loss: 1.3148 - accuracy: 0.7472 - val_loss: 0.8562 - val_accuracy: 0.8312
Epoch 2/100
16685/16685 [==============================] - 82s 5ms/step - loss: 1.0040 - accuracy: 0.7880 - val_loss: 0.7924 - val_accuracy: 0.8385
Epoch 3/100
16685/16685 [==============================] - 82s 5ms/step - loss: 0.9572 - accuracy: 0.7959 - val_loss: 0.7821 - val_accuracy: 0.8386
Epoch 4/100
16685/16685 [==============================] - 93s 6ms/step - loss: 0.9331 - accuracy: 0.8011 - val_loss: 0.7612 - val_accuracy: 0.8401
Epoch 5/100
16685/16685 [==============================] - 82s 5ms/step - loss: 0.9155 - accuracy: 0.8041 - val_loss: 0.7765 - val_accuracy: 0.8405
Epoch 6/100
16685/16685 [==============================] - 92s 6ms/step - loss: 0.9036 - accuracy: 0.8054 - val_loss: 0.7662 - val_accuracy: 0.8431
Epoch 7/100
16685/16685 [==============================] - 84s 5ms/step - loss: 0.8919 - accuracy: 0.8076 - val_

In [14]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform", kernel_regularizer="l2"),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [15]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_l2_on_FC"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_l2_on_FC_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_l2_on_FC_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
3)

Epoch 1/100
16685/16685 [==============================] - 85s 5ms/step - loss: 0.8478 - accuracy: 0.7682 - val_loss: 0.5484 - val_accuracy: 0.8328
Epoch 2/100
16685/16685 [==============================] - 83s 5ms/step - loss: 0.6611 - accuracy: 0.8088 - val_loss: 0.5208 - val_accuracy: 0.8428
Epoch 3/100
16685/16685 [==============================] - 83s 5ms/step - loss: 0.6312 - accuracy: 0.8163 - val_loss: 0.5170 - val_accuracy: 0.8428
Epoch 4/100
16685/16685 [==============================] - 93s 6ms/step - loss: 0.6152 - accuracy: 0.8210 - val_loss: 0.4981 - val_accuracy: 0.8483
Epoch 5/100
16685/16685 [==============================] - 84s 5ms/step - loss: 0.6031 - accuracy: 0.8239 - val_loss: 0.4909 - val_accuracy: 0.8500
Epoch 6/100
16685/16685 [==============================] - 83s 5ms/step - loss: 0.5936 - accuracy: 0.8265 - val_loss: 0.4888 - val_accuracy: 0.8516
Epoch 7/100
16685/16685 [==============================] - 93s 6ms/step - loss: 0.5882 - accuracy: 0.8276 - val_

# batch size

In [16]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [19]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_batch_16"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_16_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_16_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=16,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
33370/33370 [==============================] - 172s 5ms/step - loss: 0.7521 - accuracy: 0.7661 - val_loss: 0.4459 - val_accuracy: 0.8416
Epoch 2/100
33370/33370 [==============================] - 158s 5ms/step - loss: 0.5769 - accuracy: 0.8106 - val_loss: 0.4254 - val_accuracy: 0.8476
Epoch 3/100
33370/33370 [==============================] - 160s 5ms/step - loss: 0.5438 - accuracy: 0.8195 - val_loss: 0.4125 - val_accuracy: 0.8515
Epoch 4/100
33370/33370 [==============================] - 159s 5ms/step - loss: 0.5261 - accuracy: 0.8242 - val_loss: 0.4077 - val_accuracy: 0.8516
Epoch 5/100
33370/33370 [==============================] - 160s 5ms/step - loss: 0.5126 - accuracy: 0.8280 - val_loss: 0.3992 - val_accuracy: 0.8548
Epoch 6/100
33370/33370 [==============================] - 158s 5ms/step - loss: 0.5041 - accuracy: 0.8296 - val_loss: 0.3936 - val_accuracy: 0.8558
Epoch 7/100
33370/33370 [==============================] - 160s 5ms/step - loss: 0.4982 - accuracy: 0.8319

In [20]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_batch_64"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_64_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_64_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=64,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
8343/8343 [==============================] - 48s 6ms/step - loss: 0.7187 - accuracy: 0.7792 - val_loss: 0.4324 - val_accuracy: 0.8450
Epoch 2/100
8343/8343 [==============================] - 43s 5ms/step - loss: 0.5143 - accuracy: 0.8270 - val_loss: 0.4097 - val_accuracy: 0.8522
Epoch 3/100
8343/8343 [==============================] - 42s 5ms/step - loss: 0.4840 - accuracy: 0.8354 - val_loss: 0.4029 - val_accuracy: 0.8530
Epoch 4/100
8343/8343 [==============================] - 47s 6ms/step - loss: 0.4670 - accuracy: 0.8398 - val_loss: 0.3907 - val_accuracy: 0.8578
Epoch 5/100
8343/8343 [==============================] - 42s 5ms/step - loss: 0.4558 - accuracy: 0.8426 - val_loss: 0.3879 - val_accuracy: 0.8589
Epoch 6/100
8343/8343 [==============================] - 42s 5ms/step - loss: 0.4468 - accuracy: 0.8444 - val_loss: 0.3872 - val_accuracy: 0.8594
Epoch 7/100
8343/8343 [==============================] - 47s 6ms/step - loss: 0.4411 - accuracy: 0.8465 - val_loss: 0.3849 -

In [21]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_batch_128"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_128_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_128_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=128,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
4172/4172 [==============================] - 25s 6ms/step - loss: 0.7452 - accuracy: 0.7763 - val_loss: 0.4345 - val_accuracy: 0.8446
Epoch 2/100
4172/4172 [==============================] - 21s 5ms/step - loss: 0.5081 - accuracy: 0.8292 - val_loss: 0.4126 - val_accuracy: 0.8505
Epoch 3/100
4172/4172 [==============================] - 23s 6ms/step - loss: 0.4737 - accuracy: 0.8382 - val_loss: 0.4003 - val_accuracy: 0.8547
Epoch 4/100
4172/4172 [==============================] - 24s 6ms/step - loss: 0.4564 - accuracy: 0.8434 - val_loss: 0.3898 - val_accuracy: 0.8577
Epoch 5/100
4172/4172 [==============================] - 24s 6ms/step - loss: 0.4451 - accuracy: 0.8459 - val_loss: 0.3870 - val_accuracy: 0.8588
Epoch 6/100
4172/4172 [==============================] - 24s 6ms/step - loss: 0.4355 - accuracy: 0.8486 - val_loss: 0.3866 - val_accuracy: 0.8597
Epoch 7/100
4172/4172 [==============================] - 32s 8ms/step - loss: 0.4288 - accuracy: 0.8499 - val_loss: 0.3820 -

In [22]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_batch_256"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_256_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_256_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=256,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
2086/2086 [==============================] - 14s 6ms/step - loss: 0.8050 - accuracy: 0.7649 - val_loss: 0.4475 - val_accuracy: 0.8404
Epoch 2/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.5072 - accuracy: 0.8301 - val_loss: 0.4115 - val_accuracy: 0.8515
Epoch 3/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.4713 - accuracy: 0.8392 - val_loss: 0.4020 - val_accuracy: 0.8550
Epoch 4/100
2086/2086 [==============================] - 14s 7ms/step - loss: 0.4524 - accuracy: 0.8442 - val_loss: 0.3960 - val_accuracy: 0.8570
Epoch 5/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.4394 - accuracy: 0.8476 - val_loss: 0.3924 - val_accuracy: 0.8577
Epoch 6/100
2086/2086 [==============================] - 14s 7ms/step - loss: 0.4303 - accuracy: 0.8502 - val_loss: 0.3856 - val_accuracy: 0.8597
Epoch 7/100
2086/2086 [==============================] - 14s 7ms/step - loss: 0.4236 - accuracy: 0.8516 - val_loss: 0.3817 -

In [23]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_batch_512"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_512_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_batch_512_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=512,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
1043/1043 [==============================] - 10s 9ms/step - loss: 0.9355 - accuracy: 0.7394 - val_loss: 0.4670 - val_accuracy: 0.8370
Epoch 2/100
1043/1043 [==============================] - 9s 8ms/step - loss: 0.5353 - accuracy: 0.8242 - val_loss: 0.4312 - val_accuracy: 0.8466
Epoch 3/100
1043/1043 [==============================] - 8s 8ms/step - loss: 0.4893 - accuracy: 0.8355 - val_loss: 0.4111 - val_accuracy: 0.8519
Epoch 4/100
1043/1043 [==============================] - 10s 9ms/step - loss: 0.4657 - accuracy: 0.8414 - val_loss: 0.4019 - val_accuracy: 0.8545
Epoch 5/100
1043/1043 [==============================] - 11s 11ms/step - loss: 0.4510 - accuracy: 0.8450 - val_loss: 0.3926 - val_accuracy: 0.8584
Epoch 6/100
1043/1043 [==============================] - 11s 10ms/step - loss: 0.4404 - accuracy: 0.8481 - val_loss: 0.3920 - val_accuracy: 0.8580
Epoch 7/100
1043/1043 [==============================] - 9s 8ms/step - loss: 0.4326 - accuracy: 0.8497 - val_loss: 0.3859 - 

# asdf

In [4]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('tanh'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('tanh'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('tanh'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('tanh'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [6]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_tanh"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_tanh_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_tanh_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=256,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
2086/2086 [==============================] - 14s 6ms/step - loss: 0.7889 - accuracy: 0.7667 - val_loss: 0.4831 - val_accuracy: 0.8332
Epoch 2/100
2086/2086 [==============================] - 14s 7ms/step - loss: 0.5157 - accuracy: 0.8275 - val_loss: 0.4465 - val_accuracy: 0.8402
Epoch 3/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.4799 - accuracy: 0.8367 - val_loss: 0.4248 - val_accuracy: 0.8482
Epoch 4/100
2086/2086 [==============================] - 17s 8ms/step - loss: 0.4604 - accuracy: 0.8418 - val_loss: 0.4157 - val_accuracy: 0.8508
Epoch 5/100
2086/2086 [==============================] - 18s 8ms/step - loss: 0.4478 - accuracy: 0.8448 - val_loss: 0.4119 - val_accuracy: 0.8519
Epoch 6/100
2086/2086 [==============================] - 14s 7ms/step - loss: 0.4372 - accuracy: 0.8472 - val_loss: 0.4170 - val_accuracy: 0.8504
Epoch 7/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.4303 - accuracy: 0.8493 - val_loss: 0.3978 -

In [4]:
def lenet_model():
    return Sequential(
        [
            Conv2D(6, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('elu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(16, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(120, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            Flatten(),
            Dense(84, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [6]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_elu"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_elu_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_elu_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=256,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
2086/2086 [==============================] - 19s 8ms/step - loss: 0.7535 - accuracy: 0.7729 - val_loss: 0.4530 - val_accuracy: 0.8413
Epoch 2/100
2086/2086 [==============================] - 14s 7ms/step - loss: 0.5007 - accuracy: 0.8304 - val_loss: 0.4231 - val_accuracy: 0.8463
Epoch 3/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.4654 - accuracy: 0.8398 - val_loss: 0.4067 - val_accuracy: 0.8535
Epoch 4/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.4454 - accuracy: 0.8450 - val_loss: 0.3934 - val_accuracy: 0.8573
Epoch 5/100
2086/2086 [==============================] - 14s 7ms/step - loss: 0.4332 - accuracy: 0.8484 - val_loss: 0.3891 - val_accuracy: 0.8578
Epoch 6/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.4242 - accuracy: 0.8505 - val_loss: 0.3851 - val_accuracy: 0.8595
Epoch 7/100
2086/2086 [==============================] - 13s 6ms/step - loss: 0.4167 - accuracy: 0.8524 - val_loss: 0.3776 -

# asdf


In [7]:
def lenet_model():
    return Sequential(
        [
            Conv2D(12, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('elu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(32, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(240, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            Flatten(),
            Dense(162, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [9]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_x2"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_x2_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_x2_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=256,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
2086/2086 [==============================] - 18s 8ms/step - loss: 0.6222 - accuracy: 0.8015 - val_loss: 0.4256 - val_accuracy: 0.8497
Epoch 2/100
2086/2086 [==============================] - 16s 8ms/step - loss: 0.4451 - accuracy: 0.8438 - val_loss: 0.4083 - val_accuracy: 0.8527
Epoch 3/100
2086/2086 [==============================] - 16s 8ms/step - loss: 0.4159 - accuracy: 0.8516 - val_loss: 0.3867 - val_accuracy: 0.8580
Epoch 4/100
2086/2086 [==============================] - 17s 8ms/step - loss: 0.3980 - accuracy: 0.8562 - val_loss: 0.3782 - val_accuracy: 0.8611
Epoch 5/100
2086/2086 [==============================] - 15s 7ms/step - loss: 0.3859 - accuracy: 0.8594 - val_loss: 0.3695 - val_accuracy: 0.8646
Epoch 6/100
2086/2086 [==============================] - 15s 7ms/step - loss: 0.3772 - accuracy: 0.8624 - val_loss: 0.3700 - val_accuracy: 0.8634
Epoch 7/100
2086/2086 [==============================] - 15s 7ms/step - loss: 0.3686 - accuracy: 0.8644 - val_loss: 0.3605 -

In [10]:
def lenet_model():
    return Sequential(
        [
            Conv2D(24, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform", input_shape=(32, 32, 1)),
            BatchNormalization(),
            Activation('elu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(64, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            MaxPooling2D(pool_size=2, strides=2, padding="valid"),
            Conv2D(480, kernel_size=5, strides=1, padding="valid", kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            Flatten(),
            Dense(324, kernel_initializer="lecun_uniform"),
            BatchNormalization(),
            Activation('elu'),
            Dropout(0.5),
            Dense(62, activation="softmax"),
        ]
    )

In [11]:
model = lenet_model()
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/lenet_x4"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_x4_best.h5", save_best_only=True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=location + "weights/lenet_x4_last.h5", save_best_only=False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=256,
    validation_data=(X_valid_pad, y_valid),
    epochs=100,
    callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
2086/2086 [==============================] - 23s 11ms/step - loss: 0.5553 - accuracy: 0.8158 - val_loss: 0.4171 - val_accuracy: 0.8520
Epoch 2/100
2086/2086 [==============================] - 22s 11ms/step - loss: 0.4216 - accuracy: 0.8490 - val_loss: 0.4017 - val_accuracy: 0.8537
Epoch 3/100
2086/2086 [==============================] - 22s 11ms/step - loss: 0.3950 - accuracy: 0.8566 - val_loss: 0.3770 - val_accuracy: 0.8614
Epoch 4/100
2086/2086 [==============================] - 22s 11ms/step - loss: 0.3772 - accuracy: 0.8611 - val_loss: 0.3679 - val_accuracy: 0.8627
Epoch 5/100
2086/2086 [==============================] - 22s 11ms/step - loss: 0.3637 - accuracy: 0.8647 - val_loss: 0.3596 - val_accuracy: 0.8681
Epoch 6/100
2086/2086 [==============================] - 22s 10ms/step - loss: 0.3528 - accuracy: 0.8683 - val_loss: 0.3604 - val_accuracy: 0.8662
Epoch 7/100
2086/2086 [==============================] - 22s 11ms/step - loss: 0.3429 - accuracy: 0.8712 - val_loss: 0